In [85]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from os import chdir
import numpy as np
import pickle
import scipy.stats as stats


# remove worst feature

In [86]:
def clipAndNormalize(features):
    #clip the features to the range of the training data
    features.drop(['key'], axis=1, inplace=True)
    #clip outliers to 1st and 99th percentile
    features['danceability'] = features['danceability'].clip(lower=features['danceability'].quantile(0.01), upper=features['danceability'].quantile(0.99))
    features['energy'] = features['energy'].clip(lower=features['energy'].quantile(0.01), upper=features['energy'].quantile(0.99))
    features['loudness'] = features['loudness'].clip(lower=features['loudness'].quantile(0.01), upper=features['loudness'].quantile(0.99))
    features['speechiness'] = features['speechiness'].clip(lower=features['speechiness'].quantile(0.01), upper=features['speechiness'].quantile(0.99))
    features['acousticness'] = features['acousticness'].clip(lower=features['acousticness'].quantile(0.01), upper=features['acousticness'].quantile(0.99))
    features['instrumentalness'] = features['instrumentalness'].clip(lower=features['instrumentalness'].quantile(0.01), upper=features['instrumentalness'].quantile(0.99))
    features['liveness'] = features['liveness'].clip(lower=features['liveness'].quantile(0.01), upper=features['liveness'].quantile(0.99))
    features['valence'] = features['valence'].clip(lower=features['valence'].quantile(0.01), upper=features['valence'].quantile(0.99))
    features['tempo'] = features['tempo'].clip(lower=features['tempo'].quantile(0.01), upper=features['tempo'].quantile(0.99))
    features['duration_ms'] = features['duration_ms'].clip(lower=features['duration_ms'].quantile(0.01), upper=features['duration_ms'].quantile(0.99))
    features['time_signature'] = features['time_signature'].clip(lower=features['time_signature'].quantile(0.01), upper=features['time_signature'].quantile(0.99))

    columns_to_log=['liveness', 'instrumentalness', 'acousticness', 'speechiness','loudness','energy']

    for i in columns_to_log:
        if i == 'loudness':
            features[i] = features[i] + 60
        features[i] = np.log(features[i]+1)

    
    #normalize the data
    scaler = StandardScaler()

    #if id is a column, drop it
    if 'id' in features.columns:
        #fit on all columns except the track id
        rawfeatures = features.drop(['id'], axis=1)
    else:
        rawfeatures = features
    preprocessedFeatures = scaler.fit_transform(rawfeatures)

    preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    #apply z-score normalization
    for i in columns_to_log:
        preprocessedFeaturesDF[i] = stats.zscore(preprocessedFeaturesDF[i])
        preprocessedFeaturesDF.clip(lower=-2.7, upper=2.7, inplace=True)



    #preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    '''#convert to dictionary, with track id as key
    preprocessedFeatures = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)
    preprocessedFeatures['id']= features['id']
    preprocessedFeatures = preprocessedFeatures.set_index('id').T.to_dict('list')'''
    return preprocessedFeaturesDF, preprocessedFeatures

In [87]:
def makeCategorical(df):
    mood_order=['sad','angry','energetic','excited','happy','content','calm','depressed']
    mood_codes, mood_categories = pd.factorize(mood_order)
    
    # Create a categorical object with the desired order
    cat = pd.Categorical(df['mood'], categories=mood_order, ordered=True)

    # Get the integer codes of the categories
    codes = cat.codes

    # Add the codes as a new column to the dataframe
    df['mood_code'] = codes
    return df



In [88]:
chdir('C:/Users/mlar5/OneDrive/Desktop/Code Folder/Python Projects/IRL projects/Aspire - Affective Computing Project/Playlists Data/Audio Features/emotion joint data')

In [89]:
emotionsDF = pd.read_csv('Merged Emotions Data5.csv')

In [90]:
emotionsDF = makeCategorical(emotionsDF)

In [91]:
emotionsDF['mood_code'].value_counts()

7    3779
6    1218
0    1020
5     770
1     694
2     630
3     628
4     618
Name: mood_code, dtype: int64

In [92]:
# create a new df with only up to 500 songs per mood_code
# this is to balance the data

balancedDF = pd.DataFrame(columns=emotionsDF.columns)

largest_balanced_sample = emotionsDF['mood_code'].value_counts().min()

for i in emotionsDF['mood_code'].unique():
    df = emotionsDF[emotionsDF['mood_code']==i]
    #if the value count of the mood_code is larger than 500, sample 500
    if df['mood_code'].value_counts()[i] >= largest_balanced_sample:
        df = df.sample(n=largest_balanced_sample, random_state=42)
    #if the value count of the mood_code is less than 500, sample the value count
    else:
        df = df.sample(n=df['mood_code'].value_counts()[i])
    balancedDF = pd.concat([balancedDF, df])

balancedDF['mood_code'].value_counts()

1    618
6    618
5    618
7    618
2    618
3    618
4    618
0    618
Name: mood_code, dtype: int64

In [93]:
balancedDF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song,mood,genre,mood_code
381,0.848,0.520,5,-10.663,0,0.5010,0.002250,0.000799,0.679,0.304,149.996,spotify:track:2XSrt1dcuOXPgl3B4bxmBz,203897,4,Carrollton,angry,rap,1
666,0.713,0.698,10,-7.435,0,0.1680,0.180000,0.000001,0.304,0.480,124.973,spotify:track:1SSv8SA2OHfOUwLgb8yOum,180062,4,Cheat Cxdes,angry,rap,1
257,0.757,0.423,1,-2.311,1,0.0527,0.000004,0.897000,0.118,0.125,130.058,spotify:track:0A8Mrg7ButLr17K3A0R61D,133308,4,TOTALITARIANISM,angry,EDM,1
338,0.516,0.515,1,-13.005,1,0.2790,0.033600,0.000002,0.119,0.396,95.971,spotify:track:583TaS41X2JJGKoGXnTY3l,107159,4,KILLTHEPHARAOH,angry,rap,1
319,0.618,0.836,6,-4.750,0,0.0813,0.002400,0.000000,0.363,0.397,175.060,spotify:track:7CMy59461Q3pgsPZ4Cj8CP,89143,4,EASE,angry,rap,1


In [94]:
rawfeatures = balancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)

In [95]:
rawfeaturesDF, rawfeatures = clipAndNormalize(rawfeatures)

In [96]:
rawfeaturesDF.head()

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1.482419,-0.322342,-0.665392,-1.283231,2.700000,-0.958541,-0.489430,2.700000,-0.552893,0.940143,0.272427,0.173859
1,0.545890,0.371919,0.064003,-1.283231,0.657598,-0.245604,-0.492971,1.000754,0.192677,0.071470,-0.199505,0.173859
2,0.851129,-0.735754,1.136073,0.779283,-0.530916,-0.967633,2.350764,-0.477927,-1.311172,0.247995,-1.125231,0.173859
3,-0.820748,-0.342998,-1.224490,0.779283,1.695751,-0.824045,-0.492970,-0.469337,-0.163163,-0.935335,-1.642980,0.173859
4,-0.113148,0.861786,0.637965,-1.283231,-0.224387,-0.957888,-0.492977,1.425940,-0.158927,1.810239,-1.999696,0.173859


In [97]:
y = balancedDF['mood_code']

In [98]:
#set it to categorical
y = y.astype('category')

In [99]:
#X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#make a new dataframe of all the rows in emotionsDF that are not in balancedDF
unbalancedDF = emotionsDF[~emotionsDF.index.isin(balancedDF.index)]
extra_test_features = unbalancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)
extra_test_features_DF, extra_test_features = clipAndNormalize(extra_test_features)
extra_test_y = unbalancedDF['mood_code'].astype('category')
X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#combine the extra test features to the test set numpy arrays
X_test_standard = np.concatenate((X_test_standard, extra_test_features), axis=0)
y_test_standard = np.concatenate((y_test_standard, extra_test_y), axis=0)



In [100]:

# Initialize the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(256,128),random_state=42,early_stopping=True)

In [101]:
# Train the model on the resampled data
mlp.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred = mlp.predict(X_test_standard)

# Evaluate the model performance of micro-averaged F1 score

print(classification_report(y_test_standard, y_pred,digits=3))


              precision    recall  f1-score   support

           0      0.188     0.310     0.234       525
           1      0.292     0.528     0.376       199
           2      0.110     0.618     0.187       136
           3      0.071     0.507     0.125       134
           4      0.103     0.524     0.172       124
           5      0.136     0.189     0.158       275
           6      0.412     0.535     0.465       724
           7      0.747     0.114     0.198      3285

    accuracy                          0.240      5402
   macro avg      0.257     0.416     0.239      5402
weighted avg      0.552     0.240     0.239      5402



In [102]:
def offByOne(y_test_standard, y_pred,digits=3):
    #compare y_test_standard with y_pred_list. If y_pred_list is +-1 from y_test_standard, then it change it to be the same as y_test_standard
    y_test_standard_list=list(y_test_standard)
    y_pred_list = list(y_pred)
    for id in range(len(y_test_standard_list)):
        if y_test_standard_list[id] != 0 and y_test_standard_list[id] != 7:
            if y_pred_list[id] == y_test_standard_list[id] - 1 or y_pred_list[id] == y_test_standard_list[id] + 1:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 0:
            if y_pred_list[id] ==  1 or y_pred_list[id] == 7:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 7:
            if y_pred_list[id] ==  0 or y_pred_list[id] == 6:
                y_pred_list[id] = y_test_standard_list[id]
    print(classification_report(y_test_standard_list, y_pred_list,digits = digits))
    return

In [103]:
offByOne(y_test_standard, y_pred)

              precision    recall  f1-score   support

           0      0.538     0.389     0.451       525
           1      0.392     0.739     0.512       199
           2      0.145     0.787     0.244       136
           3      0.110     0.799     0.194       134
           4      0.147     0.677     0.242       124
           5      0.360     0.455     0.402       275
           6      0.940     0.756     0.838       724
           7      0.971     0.425     0.591      3285

    accuracy                          0.503      5402
   macro avg      0.450     0.628     0.434      5402
weighted avg      0.811     0.503     0.571      5402



In [104]:
#svm = SVC(kernel='linear', class_weight='balanced', random_state=42)
svm =SVC(kernel='poly', degree=3,class_weight='balanced', random_state=42)


In [105]:
# Train the model
svm.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_SVM = svm.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_SVM,digits=3))

              precision    recall  f1-score   support

           0      0.200     0.343     0.252       525
           1      0.252     0.482     0.331       199
           2      0.099     0.596     0.170       136
           3      0.076     0.336     0.124       134
           4      0.073     0.637     0.132       124
           5      0.099     0.185     0.129       275
           6      0.436     0.344     0.385       724
           7      0.742     0.125     0.213      3285

    accuracy                          0.220      5402
   macro avg      0.247     0.381     0.217      5402
weighted avg      0.550     0.220     0.235      5402



In [106]:
offByOne(y_test_standard, y_pred_SVM)

              precision    recall  f1-score   support

           0      0.533     0.427     0.474       525
           1      0.374     0.769     0.503       199
           2      0.127     0.728     0.217       136
           3      0.159     0.761     0.263       134
           4      0.090     0.694     0.159       124
           5      0.330     0.535     0.408       275
           6      0.961     0.673     0.791       724
           7      0.971     0.368     0.534      3285

    accuracy                          0.464      5402
   macro avg      0.443     0.619     0.419      5402
weighted avg      0.811     0.464     0.532      5402



In [107]:
#use a decision tree
from sklearn.tree import DecisionTreeClassifier

# Initialize the decision tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Train the model
dt.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_DT = dt.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_DT))

offByOne(y_test_standard, y_pred_DT)

              precision    recall  f1-score   support

           0       0.19      0.24      0.21       525
           1       0.18      0.46      0.26       199
           2       0.10      0.54      0.17       136
           3       0.05      0.28      0.09       134
           4       0.05      0.25      0.09       124
           5       0.10      0.21      0.13       275
           6       0.35      0.37      0.36       724
           7       0.68      0.17      0.27      3285

    accuracy                           0.23      5402
   macro avg       0.21      0.32      0.20      5402
weighted avg       0.50      0.23      0.26      5402

              precision    recall  f1-score   support

           0      0.593     0.451     0.512       525
           1      0.278     0.744     0.404       199
           2      0.143     0.713     0.238       136
           3      0.100     0.537     0.169       134
           4      0.109     0.492     0.179       124
           5      0.234 

In [108]:
# use a random forest classifier
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=1000, random_state=42)

# Train the model
rf.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_RF = rf.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_RF,digits=3))

print(offByOne(y_test_standard, y_pred_RF))

              precision    recall  f1-score   support

           0      0.217     0.381     0.276       525
           1      0.220     0.638     0.327       199
           2      0.122     0.647     0.206       136
           3      0.070     0.433     0.121       134
           4      0.123     0.500     0.197       124
           5      0.166     0.215     0.187       275
           6      0.370     0.572     0.449       724
           7      0.712     0.082     0.147      3285

    accuracy                          0.236      5402
   macro avg      0.250     0.433     0.239      5402
weighted avg      0.528     0.236     0.211      5402

              precision    recall  f1-score   support

           0      0.581     0.484     0.528       525
           1      0.294     0.854     0.437       199
           2      0.162     0.831     0.271       136
           3      0.112     0.701     0.193       134
           4      0.181     0.669     0.285       124
           5      0.378 

In [109]:
# use a bagging classifier
from sklearn.ensemble import BaggingClassifier
base_estimator = RandomForestClassifier(n_estimators=100,  random_state=42)

bagging = BaggingClassifier(base_estimator=base_estimator, n_estimators=10, random_state=42)

# Train the model
bagging.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_bagging = bagging.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_bagging,digits=3))

print(offByOne(y_test_standard, y_pred_bagging))

              precision    recall  f1-score   support

           0      0.212     0.371     0.270       525
           1      0.234     0.623     0.340       199
           2      0.114     0.662     0.195       136
           3      0.070     0.440     0.121       134
           4      0.122     0.484     0.195       124
           5      0.173     0.225     0.196       275
           6      0.373     0.579     0.454       724
           7      0.746     0.080     0.145      3285

    accuracy                          0.236      5402
   macro avg      0.256     0.433     0.240      5402
weighted avg      0.549     0.236     0.210      5402

              precision    recall  f1-score   support

           0      0.558     0.461     0.505       525
           1      0.312     0.839     0.454       199
           2      0.149     0.838     0.254       136
           3      0.112     0.716     0.194       134
           4      0.181     0.645     0.283       124
           5      0.382 

# Remove worst 2 features **BEST SO FAR**

In [135]:
def clipAndNormalize(features):
    #clip the features to the range of the training data
    features.drop(['key','time_signature'], axis=1, inplace=True)
    #clip outliers to 1st and 99th percentile
    features['danceability'] = features['danceability'].clip(lower=features['danceability'].quantile(0.01), upper=features['danceability'].quantile(0.99))
    features['energy'] = features['energy'].clip(lower=features['energy'].quantile(0.01), upper=features['energy'].quantile(0.99))
    features['loudness'] = features['loudness'].clip(lower=features['loudness'].quantile(0.01), upper=features['loudness'].quantile(0.99))
    features['speechiness'] = features['speechiness'].clip(lower=features['speechiness'].quantile(0.01), upper=features['speechiness'].quantile(0.99))
    features['acousticness'] = features['acousticness'].clip(lower=features['acousticness'].quantile(0.01), upper=features['acousticness'].quantile(0.99))
    features['instrumentalness'] = features['instrumentalness'].clip(lower=features['instrumentalness'].quantile(0.01), upper=features['instrumentalness'].quantile(0.99))
    features['liveness'] = features['liveness'].clip(lower=features['liveness'].quantile(0.01), upper=features['liveness'].quantile(0.99))
    features['valence'] = features['valence'].clip(lower=features['valence'].quantile(0.01), upper=features['valence'].quantile(0.99))
    features['tempo'] = features['tempo'].clip(lower=features['tempo'].quantile(0.01), upper=features['tempo'].quantile(0.99))
    features['duration_ms'] = features['duration_ms'].clip(lower=features['duration_ms'].quantile(0.01), upper=features['duration_ms'].quantile(0.99))
    #features['time_signature'] = features['time_signature'].clip(lower=features['time_signature'].quantile(0.01), upper=features['time_signature'].quantile(0.99))

    columns_to_log=['liveness', 'instrumentalness', 'acousticness', 'speechiness','loudness','energy']

    for i in columns_to_log:
        if i == 'loudness':
            features[i] = features[i] + 60
        features[i] = np.log(features[i]+1)

    
    #normalize the data
    scaler = StandardScaler()

    #if id is a column, drop it
    if 'id' in features.columns:
        #fit on all columns except the track id
        rawfeatures = features.drop(['id'], axis=1)
    else:
        rawfeatures = features
    preprocessedFeatures = scaler.fit_transform(rawfeatures)

    preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    #apply z-score normalization
    for i in columns_to_log:
        preprocessedFeaturesDF[i] = stats.zscore(preprocessedFeaturesDF[i])
        preprocessedFeaturesDF.clip(lower=-2.7, upper=2.7, inplace=True)



    #preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    '''#convert to dictionary, with track id as key
    preprocessedFeatures = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)
    preprocessedFeatures['id']= features['id']
    preprocessedFeatures = preprocessedFeatures.set_index('id').T.to_dict('list')'''
    return preprocessedFeaturesDF, preprocessedFeatures

In [136]:
def makeCategorical(df):
    mood_order=['sad','angry','energetic','excited','happy','content','calm','depressed']
    mood_codes, mood_categories = pd.factorize(mood_order)
    
    # Create a categorical object with the desired order
    cat = pd.Categorical(df['mood'], categories=mood_order, ordered=True)

    # Get the integer codes of the categories
    codes = cat.codes

    # Add the codes as a new column to the dataframe
    df['mood_code'] = codes
    return df



In [137]:
chdir('C:/Users/mlar5/OneDrive/Desktop/Code Folder/Python Projects/IRL projects/Aspire - Affective Computing Project/Playlists Data/Audio Features/emotion joint data')

In [138]:
emotionsDF = pd.read_csv('Merged Emotions Data5.csv')

In [139]:
emotionsDF = makeCategorical(emotionsDF)

In [140]:
emotionsDF['mood_code'].value_counts()

7    3779
6    1218
0    1020
5     770
1     694
2     630
3     628
4     618
Name: mood_code, dtype: int64

In [141]:
# create a new df with only up to 500 songs per mood_code
# this is to balance the data

balancedDF = pd.DataFrame(columns=emotionsDF.columns)

largest_balanced_sample = emotionsDF['mood_code'].value_counts().min()

for i in emotionsDF['mood_code'].unique():
    df = emotionsDF[emotionsDF['mood_code']==i]
    #if the value count of the mood_code is larger than 500, sample 500
    if df['mood_code'].value_counts()[i] >= largest_balanced_sample:
        df = df.sample(n=largest_balanced_sample, random_state=42)
    #if the value count of the mood_code is less than 500, sample the value count
    else:
        df = df.sample(n=df['mood_code'].value_counts()[i])
    balancedDF = pd.concat([balancedDF, df])

balancedDF['mood_code'].value_counts()

1    618
6    618
5    618
7    618
2    618
3    618
4    618
0    618
Name: mood_code, dtype: int64

In [142]:
balancedDF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song,mood,genre,mood_code
381,0.848,0.520,5,-10.663,0,0.5010,0.002250,0.000799,0.679,0.304,149.996,spotify:track:2XSrt1dcuOXPgl3B4bxmBz,203897,4,Carrollton,angry,rap,1
666,0.713,0.698,10,-7.435,0,0.1680,0.180000,0.000001,0.304,0.480,124.973,spotify:track:1SSv8SA2OHfOUwLgb8yOum,180062,4,Cheat Cxdes,angry,rap,1
257,0.757,0.423,1,-2.311,1,0.0527,0.000004,0.897000,0.118,0.125,130.058,spotify:track:0A8Mrg7ButLr17K3A0R61D,133308,4,TOTALITARIANISM,angry,EDM,1
338,0.516,0.515,1,-13.005,1,0.2790,0.033600,0.000002,0.119,0.396,95.971,spotify:track:583TaS41X2JJGKoGXnTY3l,107159,4,KILLTHEPHARAOH,angry,rap,1
319,0.618,0.836,6,-4.750,0,0.0813,0.002400,0.000000,0.363,0.397,175.060,spotify:track:7CMy59461Q3pgsPZ4Cj8CP,89143,4,EASE,angry,rap,1


In [143]:
rawfeatures = balancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)

In [144]:
rawfeaturesDF, rawfeatures = clipAndNormalize(rawfeatures)

In [145]:
rawfeaturesDF.head()

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,1.482419,-0.322342,-0.665392,-1.283231,2.700000,-0.958541,-0.489430,2.700000,-0.552893,0.940143,0.272427
1,0.545890,0.371919,0.064003,-1.283231,0.657598,-0.245604,-0.492971,1.000754,0.192677,0.071470,-0.199505
2,0.851129,-0.735754,1.136073,0.779283,-0.530916,-0.967633,2.350764,-0.477927,-1.311172,0.247995,-1.125231
3,-0.820748,-0.342998,-1.224490,0.779283,1.695751,-0.824045,-0.492970,-0.469337,-0.163163,-0.935335,-1.642980
4,-0.113148,0.861786,0.637965,-1.283231,-0.224387,-0.957888,-0.492977,1.425940,-0.158927,1.810239,-1.999696


In [146]:
y = balancedDF['mood_code']

In [147]:
#set it to categorical
y = y.astype('category')

In [148]:
#X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#make a new dataframe of all the rows in emotionsDF that are not in balancedDF
unbalancedDF = emotionsDF[~emotionsDF.index.isin(balancedDF.index)]
extra_test_features = unbalancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)
extra_test_features_DF, extra_test_features = clipAndNormalize(extra_test_features)
extra_test_y = unbalancedDF['mood_code'].astype('category')
X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#combine the extra test features to the test set numpy arrays
X_test_standard = np.concatenate((X_test_standard, extra_test_features), axis=0)
y_test_standard = np.concatenate((y_test_standard, extra_test_y), axis=0)



In [149]:

# Initialize the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(256,128),random_state=42,early_stopping=True)

In [150]:
# Train the model on the resampled data
mlp.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred = mlp.predict(X_test_standard)

# Evaluate the model performance of micro-averaged F1 score

print(classification_report(y_test_standard, y_pred,digits=3))


              precision    recall  f1-score   support

           0      0.210     0.343     0.261       525
           1      0.257     0.563     0.353       199
           2      0.111     0.596     0.187       136
           3      0.076     0.463     0.130       134
           4      0.099     0.508     0.165       124
           5      0.146     0.265     0.188       275
           6      0.419     0.511     0.460       724
           7      0.766     0.125     0.216      3285

    accuracy                          0.250      5402
   macro avg      0.260     0.422     0.245      5402
weighted avg      0.566     0.250     0.252      5402



In [151]:
def offByOne(y_test_standard, y_pred,digits=3):
    #compare y_test_standard with y_pred_list. If y_pred_list is +-1 from y_test_standard, then it change it to be the same as y_test_standard
    y_test_standard_list=list(y_test_standard)
    y_pred_list = list(y_pred)
    for id in range(len(y_test_standard_list)):
        if y_test_standard_list[id] != 0 and y_test_standard_list[id] != 7:
            if y_pred_list[id] == y_test_standard_list[id] - 1 or y_pred_list[id] == y_test_standard_list[id] + 1:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 0:
            if y_pred_list[id] ==  1 or y_pred_list[id] == 7:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 7:
            if y_pred_list[id] ==  0 or y_pred_list[id] == 6:
                y_pred_list[id] = y_test_standard_list[id]
    print(classification_report(y_test_standard_list, y_pred_list,digits = digits))
    return

In [152]:
offByOne(y_test_standard, y_pred)

              precision    recall  f1-score   support

           0      0.574     0.429     0.491       525
           1      0.349     0.784     0.483       199
           2      0.144     0.750     0.242       136
           3      0.119     0.746     0.206       134
           4      0.144     0.669     0.237       124
           5      0.323     0.513     0.397       275
           6      0.961     0.757     0.847       724
           7      0.969     0.423     0.589      3285

    accuracy                          0.508      5402
   macro avg      0.448     0.634     0.436      5402
weighted avg      0.813     0.508     0.574      5402



In [153]:
#svm = SVC(kernel='linear', class_weight='balanced', random_state=42)
svm =SVC(kernel='poly', degree=3,class_weight='balanced', random_state=42)


In [154]:
# Train the model
svm.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_SVM = svm.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_SVM,digits=3))

              precision    recall  f1-score   support

           0      0.205     0.387     0.268       525
           1      0.258     0.487     0.337       199
           2      0.103     0.625     0.177       136
           3      0.080     0.366     0.131       134
           4      0.076     0.605     0.136       124
           5      0.098     0.193     0.130       275
           6      0.432     0.372     0.400       724
           7      0.741     0.102     0.179      3285

    accuracy                          0.216      5402
   macro avg      0.249     0.392     0.220      5402
weighted avg      0.549     0.216     0.218      5402



In [155]:
offByOne(y_test_standard, y_pred_SVM)

              precision    recall  f1-score   support

           0      0.537     0.453     0.492       525
           1      0.379     0.769     0.507       199
           2      0.131     0.757     0.224       136
           3      0.154     0.754     0.255       134
           4      0.096     0.677     0.168       124
           5      0.307     0.516     0.385       275
           6      0.964     0.675     0.794       724
           7      0.970     0.374     0.540      3285

    accuracy                          0.470      5402
   macro avg      0.442     0.622     0.421      5402
weighted avg      0.810     0.470     0.537      5402



In [156]:
#use a decision tree
from sklearn.tree import DecisionTreeClassifier

# Initialize the decision tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Train the model
dt.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_DT = dt.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_DT))

offByOne(y_test_standard, y_pred_DT)

              precision    recall  f1-score   support

           0       0.18      0.23      0.20       525
           1       0.19      0.51      0.28       199
           2       0.10      0.52      0.17       136
           3       0.06      0.29      0.09       134
           4       0.05      0.24      0.09       124
           5       0.10      0.23      0.13       275
           6       0.36      0.37      0.37       724
           7       0.68      0.17      0.27      3285

    accuracy                           0.23      5402
   macro avg       0.21      0.32      0.20      5402
weighted avg       0.50      0.23      0.26      5402

              precision    recall  f1-score   support

           0      0.591     0.444     0.507       525
           1      0.288     0.764     0.419       199
           2      0.148     0.750     0.248       136
           3      0.105     0.545     0.176       134
           4      0.114     0.492     0.185       124
           5      0.231 

In [157]:
# use a random forest classifier
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=1000, random_state=42)

# Train the model
rf.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_RF = rf.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_RF,digits=3))

print(offByOne(y_test_standard, y_pred_RF))

              precision    recall  f1-score   support

           0      0.218     0.387     0.279       525
           1      0.219     0.613     0.322       199
           2      0.122     0.654     0.206       136
           3      0.071     0.440     0.122       134
           4      0.129     0.516     0.207       124
           5      0.160     0.200     0.178       275
           6      0.371     0.575     0.451       724
           7      0.704     0.084     0.150      3285

    accuracy                          0.238      5402
   macro avg      0.249     0.434     0.239      5402
weighted avg      0.523     0.238     0.213      5402

              precision    recall  f1-score   support

           0      0.591     0.493     0.538       525
           1      0.294     0.824     0.433       199
           2      0.162     0.838     0.271       136
           3      0.112     0.709     0.193       134
           4      0.188     0.685     0.296       124
           5      0.385 

In [158]:
# use a bagging classifier
from sklearn.ensemble import BaggingClassifier
base_estimator = RandomForestClassifier(n_estimators=100,  random_state=42)

bagging = BaggingClassifier(base_estimator=base_estimator, n_estimators=10, random_state=42)

# Train the model
bagging.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_bagging = bagging.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_bagging,digits=3))

print(offByOne(y_test_standard, y_pred_bagging))

              precision    recall  f1-score   support

           0      0.213     0.371     0.271       525
           1      0.233     0.623     0.339       199
           2      0.118     0.669     0.200       136
           3      0.071     0.448     0.122       134
           4      0.123     0.492     0.197       124
           5      0.169     0.215     0.189       275
           6      0.376     0.586     0.458       724
           7      0.721     0.079     0.142      3285

    accuracy                          0.236      5402
   macro avg      0.253     0.435     0.240      5402
weighted avg      0.534     0.236     0.209      5402

              precision    recall  f1-score   support

           0      0.568     0.463     0.510       525
           1      0.310     0.839     0.453       199
           2      0.154     0.846     0.260       136
           3      0.113     0.731     0.196       134
           4      0.188     0.677     0.294       124
           5      0.386 

# Remove worst 3 features

In [159]:
def clipAndNormalize(features):
    #clip the features to the range of the training data
    features.drop(['key','time_signature','mode'], axis=1, inplace=True)
    #clip outliers to 1st and 99th percentile
    features['danceability'] = features['danceability'].clip(lower=features['danceability'].quantile(0.01), upper=features['danceability'].quantile(0.99))
    features['energy'] = features['energy'].clip(lower=features['energy'].quantile(0.01), upper=features['energy'].quantile(0.99))
    features['loudness'] = features['loudness'].clip(lower=features['loudness'].quantile(0.01), upper=features['loudness'].quantile(0.99))
    features['speechiness'] = features['speechiness'].clip(lower=features['speechiness'].quantile(0.01), upper=features['speechiness'].quantile(0.99))
    features['acousticness'] = features['acousticness'].clip(lower=features['acousticness'].quantile(0.01), upper=features['acousticness'].quantile(0.99))
    features['instrumentalness'] = features['instrumentalness'].clip(lower=features['instrumentalness'].quantile(0.01), upper=features['instrumentalness'].quantile(0.99))
    features['liveness'] = features['liveness'].clip(lower=features['liveness'].quantile(0.01), upper=features['liveness'].quantile(0.99))
    features['valence'] = features['valence'].clip(lower=features['valence'].quantile(0.01), upper=features['valence'].quantile(0.99))
    features['tempo'] = features['tempo'].clip(lower=features['tempo'].quantile(0.01), upper=features['tempo'].quantile(0.99))
    features['duration_ms'] = features['duration_ms'].clip(lower=features['duration_ms'].quantile(0.01), upper=features['duration_ms'].quantile(0.99))
    #features['time_signature'] = features['time_signature'].clip(lower=features['time_signature'].quantile(0.01), upper=features['time_signature'].quantile(0.99))

    columns_to_log=['liveness', 'instrumentalness', 'acousticness', 'speechiness','loudness','energy']

    for i in columns_to_log:
        if i == 'loudness':
            features[i] = features[i] + 60
        features[i] = np.log(features[i]+1)

    
    #normalize the data
    scaler = StandardScaler()

    #if id is a column, drop it
    if 'id' in features.columns:
        #fit on all columns except the track id
        rawfeatures = features.drop(['id'], axis=1)
    else:
        rawfeatures = features
    preprocessedFeatures = scaler.fit_transform(rawfeatures)

    preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    #apply z-score normalization
    for i in columns_to_log:
        preprocessedFeaturesDF[i] = stats.zscore(preprocessedFeaturesDF[i])
        preprocessedFeaturesDF.clip(lower=-2.7, upper=2.7, inplace=True)



    #preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    '''#convert to dictionary, with track id as key
    preprocessedFeatures = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)
    preprocessedFeatures['id']= features['id']
    preprocessedFeatures = preprocessedFeatures.set_index('id').T.to_dict('list')'''
    return preprocessedFeaturesDF, preprocessedFeatures

In [160]:
def makeCategorical(df):
    mood_order=['sad','angry','energetic','excited','happy','content','calm','depressed']
    mood_codes, mood_categories = pd.factorize(mood_order)
    
    # Create a categorical object with the desired order
    cat = pd.Categorical(df['mood'], categories=mood_order, ordered=True)

    # Get the integer codes of the categories
    codes = cat.codes

    # Add the codes as a new column to the dataframe
    df['mood_code'] = codes
    return df



In [161]:
chdir('C:/Users/mlar5/OneDrive/Desktop/Code Folder/Python Projects/IRL projects/Aspire - Affective Computing Project/Playlists Data/Audio Features/emotion joint data')

In [162]:
emotionsDF = pd.read_csv('Merged Emotions Data5.csv')

In [163]:
emotionsDF = makeCategorical(emotionsDF)

In [164]:
emotionsDF['mood_code'].value_counts()

7    3779
6    1218
0    1020
5     770
1     694
2     630
3     628
4     618
Name: mood_code, dtype: int64

In [165]:
# create a new df with only up to 500 songs per mood_code
# this is to balance the data

balancedDF = pd.DataFrame(columns=emotionsDF.columns)

largest_balanced_sample = emotionsDF['mood_code'].value_counts().min()

for i in emotionsDF['mood_code'].unique():
    df = emotionsDF[emotionsDF['mood_code']==i]
    #if the value count of the mood_code is larger than 500, sample 500
    if df['mood_code'].value_counts()[i] >= largest_balanced_sample:
        df = df.sample(n=largest_balanced_sample, random_state=42)
    #if the value count of the mood_code is less than 500, sample the value count
    else:
        df = df.sample(n=df['mood_code'].value_counts()[i])
    balancedDF = pd.concat([balancedDF, df])

balancedDF['mood_code'].value_counts()

1    618
6    618
5    618
7    618
2    618
3    618
4    618
0    618
Name: mood_code, dtype: int64

In [166]:
balancedDF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song,mood,genre,mood_code
381,0.848,0.520,5,-10.663,0,0.5010,0.002250,0.000799,0.679,0.304,149.996,spotify:track:2XSrt1dcuOXPgl3B4bxmBz,203897,4,Carrollton,angry,rap,1
666,0.713,0.698,10,-7.435,0,0.1680,0.180000,0.000001,0.304,0.480,124.973,spotify:track:1SSv8SA2OHfOUwLgb8yOum,180062,4,Cheat Cxdes,angry,rap,1
257,0.757,0.423,1,-2.311,1,0.0527,0.000004,0.897000,0.118,0.125,130.058,spotify:track:0A8Mrg7ButLr17K3A0R61D,133308,4,TOTALITARIANISM,angry,EDM,1
338,0.516,0.515,1,-13.005,1,0.2790,0.033600,0.000002,0.119,0.396,95.971,spotify:track:583TaS41X2JJGKoGXnTY3l,107159,4,KILLTHEPHARAOH,angry,rap,1
319,0.618,0.836,6,-4.750,0,0.0813,0.002400,0.000000,0.363,0.397,175.060,spotify:track:7CMy59461Q3pgsPZ4Cj8CP,89143,4,EASE,angry,rap,1


In [167]:
rawfeatures = balancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)

In [168]:
rawfeaturesDF, rawfeatures = clipAndNormalize(rawfeatures)

In [169]:
rawfeaturesDF.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,1.482419,-0.322342,-0.665392,2.700000,-0.958541,-0.489430,2.700000,-0.552893,0.940143,0.272427
1,0.545890,0.371919,0.064003,0.657598,-0.245604,-0.492971,1.000754,0.192677,0.071470,-0.199505
2,0.851129,-0.735754,1.136073,-0.530916,-0.967633,2.350764,-0.477927,-1.311172,0.247995,-1.125231
3,-0.820748,-0.342998,-1.224490,1.695751,-0.824045,-0.492970,-0.469337,-0.163163,-0.935335,-1.642980
4,-0.113148,0.861786,0.637965,-0.224387,-0.957888,-0.492977,1.425940,-0.158927,1.810239,-1.999696


In [170]:
y = balancedDF['mood_code']

In [171]:
#set it to categorical
y = y.astype('category')

In [172]:
#X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#make a new dataframe of all the rows in emotionsDF that are not in balancedDF
unbalancedDF = emotionsDF[~emotionsDF.index.isin(balancedDF.index)]
extra_test_features = unbalancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)
extra_test_features_DF, extra_test_features = clipAndNormalize(extra_test_features)
extra_test_y = unbalancedDF['mood_code'].astype('category')
X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#combine the extra test features to the test set numpy arrays
X_test_standard = np.concatenate((X_test_standard, extra_test_features), axis=0)
y_test_standard = np.concatenate((y_test_standard, extra_test_y), axis=0)



In [173]:

# Initialize the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(256,128),random_state=42,early_stopping=True)

In [174]:
# Train the model on the resampled data
mlp.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred = mlp.predict(X_test_standard)

# Evaluate the model performance of micro-averaged F1 score

print(classification_report(y_test_standard, y_pred,digits=3))


              precision    recall  f1-score   support

           0      0.193     0.354     0.249       525
           1      0.223     0.633     0.330       199
           2      0.111     0.654     0.190       136
           3      0.071     0.418     0.121       134
           4      0.102     0.395     0.162       124
           5      0.180     0.193     0.186       275
           6      0.400     0.557     0.465       724
           7      0.741     0.112     0.194      3285

    accuracy                          0.246      5402
   macro avg      0.253     0.415     0.237      5402
weighted avg      0.547     0.246     0.238      5402



In [175]:
def offByOne(y_test_standard, y_pred,digits=3):
    #compare y_test_standard with y_pred_list. If y_pred_list is +-1 from y_test_standard, then it change it to be the same as y_test_standard
    y_test_standard_list=list(y_test_standard)
    y_pred_list = list(y_pred)
    for id in range(len(y_test_standard_list)):
        if y_test_standard_list[id] != 0 and y_test_standard_list[id] != 7:
            if y_pred_list[id] == y_test_standard_list[id] - 1 or y_pred_list[id] == y_test_standard_list[id] + 1:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 0:
            if y_pred_list[id] ==  1 or y_pred_list[id] == 7:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 7:
            if y_pred_list[id] ==  0 or y_pred_list[id] == 6:
                y_pred_list[id] = y_test_standard_list[id]
    print(classification_report(y_test_standard_list, y_pred_list,digits = digits))
    return

In [176]:
offByOne(y_test_standard, y_pred)

              precision    recall  f1-score   support

           0      0.535     0.453     0.491       525
           1      0.297     0.839     0.438       199
           2      0.147     0.846     0.251       136
           3      0.115     0.694     0.197       134
           4      0.156     0.548     0.243       124
           5      0.406     0.400     0.403       275
           6      0.940     0.736     0.826       724
           7      0.970     0.452     0.617      3285

    accuracy                          0.520      5402
   macro avg      0.446     0.621     0.433      5402
weighted avg      0.810     0.520     0.587      5402



In [177]:
#svm = SVC(kernel='linear', class_weight='balanced', random_state=42)
svm =SVC(kernel='poly', degree=3,class_weight='balanced', random_state=42)


In [178]:
# Train the model
svm.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_SVM = svm.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_SVM,digits=3))

              precision    recall  f1-score   support

           0      0.210     0.370     0.267       525
           1      0.266     0.452     0.335       199
           2      0.099     0.588     0.169       136
           3      0.082     0.403     0.136       134
           4      0.072     0.653     0.130       124
           5      0.100     0.196     0.132       275
           6      0.453     0.314     0.371       724
           7      0.767     0.118     0.205      3285

    accuracy                          0.216      5402
   macro avg      0.256     0.387     0.218      5402
weighted avg      0.569     0.216     0.230      5402



In [179]:
offByOne(y_test_standard, y_pred_SVM)

              precision    recall  f1-score   support

           0      0.528     0.427     0.472       525
           1      0.411     0.764     0.534       199
           2      0.128     0.721     0.218       136
           3      0.152     0.791     0.255       134
           4      0.093     0.750     0.165       124
           5      0.319     0.542     0.402       275
           6      0.967     0.655     0.781       724
           7      0.980     0.353     0.519      3285

    accuracy                          0.455      5402
   macro avg      0.447     0.625     0.418      5402
weighted avg      0.817     0.455     0.522      5402



In [180]:
#use a decision tree
from sklearn.tree import DecisionTreeClassifier

# Initialize the decision tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Train the model
dt.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_DT = dt.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_DT))

offByOne(y_test_standard, y_pred_DT)

              precision    recall  f1-score   support

           0       0.19      0.24      0.21       525
           1       0.18      0.47      0.26       199
           2       0.10      0.54      0.17       136
           3       0.05      0.25      0.08       134
           4       0.06      0.27      0.10       124
           5       0.09      0.21      0.13       275
           6       0.35      0.36      0.36       724
           7       0.67      0.17      0.27      3285

    accuracy                           0.23      5402
   macro avg       0.21      0.31      0.20      5402
weighted avg       0.49      0.23      0.26      5402

              precision    recall  f1-score   support

           0      0.593     0.448     0.510       525
           1      0.278     0.719     0.401       199
           2      0.146     0.757     0.244       136
           3      0.100     0.522     0.168       134
           4      0.120     0.524     0.195       124
           5      0.226 

In [181]:
# use a random forest classifier
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=1000, random_state=42)

# Train the model
rf.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_RF = rf.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_RF,digits=3))

print(offByOne(y_test_standard, y_pred_RF))

              precision    recall  f1-score   support

           0      0.213     0.375     0.272       525
           1      0.220     0.638     0.327       199
           2      0.121     0.640     0.204       136
           3      0.070     0.425     0.120       134
           4      0.122     0.500     0.196       124
           5      0.155     0.193     0.172       275
           6      0.373     0.580     0.454       724
           7      0.721     0.086     0.153      3285

    accuracy                          0.238      5402
   macro avg      0.249     0.430     0.237      5402
weighted avg      0.532     0.238     0.214      5402

              precision    recall  f1-score   support

           0      0.571     0.482     0.523       525
           1      0.294     0.849     0.437       199
           2      0.161     0.824     0.269       136
           3      0.112     0.694     0.193       134
           4      0.178     0.669     0.281       124
           5      0.375 

In [182]:
# use a bagging classifier
from sklearn.ensemble import BaggingClassifier
base_estimator = RandomForestClassifier(n_estimators=100,  random_state=42)

bagging = BaggingClassifier(base_estimator=base_estimator, n_estimators=10, random_state=42)

# Train the model
bagging.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_bagging = bagging.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_bagging,digits=3))

print(offByOne(y_test_standard, y_pred_bagging))

              precision    recall  f1-score   support

           0      0.215     0.377     0.274       525
           1      0.229     0.628     0.336       199
           2      0.116     0.654     0.197       136
           3      0.073     0.455     0.125       134
           4      0.127     0.492     0.202       124
           5      0.166     0.207     0.184       275
           6      0.377     0.586     0.458       724
           7      0.726     0.084     0.150      3285

    accuracy                          0.239      5402
   macro avg      0.253     0.435     0.241      5402
weighted avg      0.537     0.239     0.214      5402

              precision    recall  f1-score   support

           0      0.570     0.472     0.517       525
           1      0.304     0.844     0.447       199
           2      0.151     0.824     0.255       136
           3      0.112     0.709     0.193       134
           4      0.188     0.669     0.293       124
           5      0.368 

# Remove worst 4 features

In [183]:
def clipAndNormalize(features):
    #clip the features to the range of the training data
    features.drop(['key','time_signature','mode', 'tempo'], axis=1, inplace=True)
    #clip outliers to 1st and 99th percentile
    features['danceability'] = features['danceability'].clip(lower=features['danceability'].quantile(0.01), upper=features['danceability'].quantile(0.99))
    features['energy'] = features['energy'].clip(lower=features['energy'].quantile(0.01), upper=features['energy'].quantile(0.99))
    features['loudness'] = features['loudness'].clip(lower=features['loudness'].quantile(0.01), upper=features['loudness'].quantile(0.99))
    features['speechiness'] = features['speechiness'].clip(lower=features['speechiness'].quantile(0.01), upper=features['speechiness'].quantile(0.99))
    features['acousticness'] = features['acousticness'].clip(lower=features['acousticness'].quantile(0.01), upper=features['acousticness'].quantile(0.99))
    features['instrumentalness'] = features['instrumentalness'].clip(lower=features['instrumentalness'].quantile(0.01), upper=features['instrumentalness'].quantile(0.99))
    features['liveness'] = features['liveness'].clip(lower=features['liveness'].quantile(0.01), upper=features['liveness'].quantile(0.99))
    features['valence'] = features['valence'].clip(lower=features['valence'].quantile(0.01), upper=features['valence'].quantile(0.99))
    #features['tempo'] = features['tempo'].clip(lower=features['tempo'].quantile(0.01), upper=features['tempo'].quantile(0.99))
    features['duration_ms'] = features['duration_ms'].clip(lower=features['duration_ms'].quantile(0.01), upper=features['duration_ms'].quantile(0.99))
    #features['time_signature'] = features['time_signature'].clip(lower=features['time_signature'].quantile(0.01), upper=features['time_signature'].quantile(0.99))

    columns_to_log=['liveness', 'instrumentalness', 'acousticness', 'speechiness','loudness','energy']

    for i in columns_to_log:
        if i == 'loudness':
            features[i] = features[i] + 60
        features[i] = np.log(features[i]+1)

    
    #normalize the data
    scaler = StandardScaler()

    #if id is a column, drop it
    if 'id' in features.columns:
        #fit on all columns except the track id
        rawfeatures = features.drop(['id'], axis=1)
    else:
        rawfeatures = features
    preprocessedFeatures = scaler.fit_transform(rawfeatures)

    preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    #apply z-score normalization
    for i in columns_to_log:
        preprocessedFeaturesDF[i] = stats.zscore(preprocessedFeaturesDF[i])
        preprocessedFeaturesDF.clip(lower=-2.7, upper=2.7, inplace=True)



    #preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    '''#convert to dictionary, with track id as key
    preprocessedFeatures = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)
    preprocessedFeatures['id']= features['id']
    preprocessedFeatures = preprocessedFeatures.set_index('id').T.to_dict('list')'''
    return preprocessedFeaturesDF, preprocessedFeatures

In [184]:
def makeCategorical(df):
    mood_order=['sad','angry','energetic','excited','happy','content','calm','depressed']
    mood_codes, mood_categories = pd.factorize(mood_order)
    
    # Create a categorical object with the desired order
    cat = pd.Categorical(df['mood'], categories=mood_order, ordered=True)

    # Get the integer codes of the categories
    codes = cat.codes

    # Add the codes as a new column to the dataframe
    df['mood_code'] = codes
    return df



In [185]:
chdir('C:/Users/mlar5/OneDrive/Desktop/Code Folder/Python Projects/IRL projects/Aspire - Affective Computing Project/Playlists Data/Audio Features/emotion joint data')

In [186]:
emotionsDF = pd.read_csv('Merged Emotions Data5.csv')

In [187]:
emotionsDF = makeCategorical(emotionsDF)

In [188]:
emotionsDF['mood_code'].value_counts()

7    3779
6    1218
0    1020
5     770
1     694
2     630
3     628
4     618
Name: mood_code, dtype: int64

In [189]:
# create a new df with only up to 500 songs per mood_code
# this is to balance the data

balancedDF = pd.DataFrame(columns=emotionsDF.columns)

largest_balanced_sample = emotionsDF['mood_code'].value_counts().min()

for i in emotionsDF['mood_code'].unique():
    df = emotionsDF[emotionsDF['mood_code']==i]
    #if the value count of the mood_code is larger than 500, sample 500
    if df['mood_code'].value_counts()[i] >= largest_balanced_sample:
        df = df.sample(n=largest_balanced_sample, random_state=42)
    #if the value count of the mood_code is less than 500, sample the value count
    else:
        df = df.sample(n=df['mood_code'].value_counts()[i])
    balancedDF = pd.concat([balancedDF, df])

balancedDF['mood_code'].value_counts()

1    618
6    618
5    618
7    618
2    618
3    618
4    618
0    618
Name: mood_code, dtype: int64

In [190]:
balancedDF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song,mood,genre,mood_code
381,0.848,0.520,5,-10.663,0,0.5010,0.002250,0.000799,0.679,0.304,149.996,spotify:track:2XSrt1dcuOXPgl3B4bxmBz,203897,4,Carrollton,angry,rap,1
666,0.713,0.698,10,-7.435,0,0.1680,0.180000,0.000001,0.304,0.480,124.973,spotify:track:1SSv8SA2OHfOUwLgb8yOum,180062,4,Cheat Cxdes,angry,rap,1
257,0.757,0.423,1,-2.311,1,0.0527,0.000004,0.897000,0.118,0.125,130.058,spotify:track:0A8Mrg7ButLr17K3A0R61D,133308,4,TOTALITARIANISM,angry,EDM,1
338,0.516,0.515,1,-13.005,1,0.2790,0.033600,0.000002,0.119,0.396,95.971,spotify:track:583TaS41X2JJGKoGXnTY3l,107159,4,KILLTHEPHARAOH,angry,rap,1
319,0.618,0.836,6,-4.750,0,0.0813,0.002400,0.000000,0.363,0.397,175.060,spotify:track:7CMy59461Q3pgsPZ4Cj8CP,89143,4,EASE,angry,rap,1


In [191]:
rawfeatures = balancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)

In [192]:
rawfeaturesDF, rawfeatures = clipAndNormalize(rawfeatures)

In [193]:
rawfeaturesDF.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms
0,1.482419,-0.322342,-0.665392,2.700000,-0.958541,-0.489430,2.700000,-0.552893,0.272427
1,0.545890,0.371919,0.064003,0.657598,-0.245604,-0.492971,1.000754,0.192677,-0.199505
2,0.851129,-0.735754,1.136073,-0.530916,-0.967633,2.350764,-0.477927,-1.311172,-1.125231
3,-0.820748,-0.342998,-1.224490,1.695751,-0.824045,-0.492970,-0.469337,-0.163163,-1.642980
4,-0.113148,0.861786,0.637965,-0.224387,-0.957888,-0.492977,1.425940,-0.158927,-1.999696


In [194]:
y = balancedDF['mood_code']

In [195]:
#set it to categorical
y = y.astype('category')

In [196]:
#X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#make a new dataframe of all the rows in emotionsDF that are not in balancedDF
unbalancedDF = emotionsDF[~emotionsDF.index.isin(balancedDF.index)]
extra_test_features = unbalancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)
extra_test_features_DF, extra_test_features = clipAndNormalize(extra_test_features)
extra_test_y = unbalancedDF['mood_code'].astype('category')
X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#combine the extra test features to the test set numpy arrays
X_test_standard = np.concatenate((X_test_standard, extra_test_features), axis=0)
y_test_standard = np.concatenate((y_test_standard, extra_test_y), axis=0)



In [197]:

# Initialize the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(256,128),random_state=42,early_stopping=True)

In [198]:
# Train the model on the resampled data
mlp.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred = mlp.predict(X_test_standard)

# Evaluate the model performance of micro-averaged F1 score

print(classification_report(y_test_standard, y_pred,digits=3))


              precision    recall  f1-score   support

           0      0.215     0.349     0.266       525
           1      0.288     0.492     0.364       199
           2      0.106     0.618     0.181       136
           3      0.070     0.455     0.122       134
           4      0.086     0.460     0.146       124
           5      0.174     0.200     0.186       275
           6      0.394     0.622     0.482       724
           7      0.747     0.099     0.175      3285

    accuracy                          0.243      5402
   macro avg      0.260     0.412     0.240      5402
weighted avg      0.554     0.243     0.231      5402



In [199]:
def offByOne(y_test_standard, y_pred,digits=3):
    #compare y_test_standard with y_pred_list. If y_pred_list is +-1 from y_test_standard, then it change it to be the same as y_test_standard
    y_test_standard_list=list(y_test_standard)
    y_pred_list = list(y_pred)
    for id in range(len(y_test_standard_list)):
        if y_test_standard_list[id] != 0 and y_test_standard_list[id] != 7:
            if y_pred_list[id] == y_test_standard_list[id] - 1 or y_pred_list[id] == y_test_standard_list[id] + 1:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 0:
            if y_pred_list[id] ==  1 or y_pred_list[id] == 7:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 7:
            if y_pred_list[id] ==  0 or y_pred_list[id] == 6:
                y_pred_list[id] = y_test_standard_list[id]
    print(classification_report(y_test_standard_list, y_pred_list,digits = digits))
    return

In [200]:
offByOne(y_test_standard, y_pred)

              precision    recall  f1-score   support

           0      0.570     0.413     0.479       525
           1      0.411     0.764     0.534       199
           2      0.138     0.765     0.234       136
           3      0.121     0.799     0.210       134
           4      0.136     0.645     0.225       124
           5      0.412     0.509     0.455       275
           6      0.940     0.753     0.836       724
           7      0.967     0.442     0.607      3285

    accuracy                          0.518      5402
   macro avg      0.462     0.636     0.447      5402
weighted avg      0.815     0.518     0.587      5402



In [201]:
#svm = SVC(kernel='linear', class_weight='balanced', random_state=42)
svm =SVC(kernel='poly', degree=3,class_weight='balanced', random_state=42)


In [202]:
# Train the model
svm.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_SVM = svm.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_SVM,digits=3))

              precision    recall  f1-score   support

           0      0.189     0.333     0.242       525
           1      0.270     0.482     0.347       199
           2      0.093     0.574     0.159       136
           3      0.078     0.373     0.130       134
           4      0.069     0.637     0.125       124
           5      0.095     0.167     0.121       275
           6      0.493     0.304     0.376       724
           7      0.771     0.135     0.229      3285

    accuracy                          0.220      5402
   macro avg      0.257     0.376     0.216      5402
weighted avg      0.574     0.220     0.242      5402



In [203]:
offByOne(y_test_standard, y_pred_SVM)

              precision    recall  f1-score   support

           0      0.481     0.370     0.418       525
           1      0.402     0.774     0.529       199
           2      0.127     0.750     0.217       136
           3      0.153     0.776     0.256       134
           4      0.085     0.702     0.152       124
           5      0.336     0.524     0.409       275
           6      0.957     0.642     0.769       724
           7      0.977     0.356     0.522      3285

    accuracy                          0.448      5402
   macro avg      0.440     0.612     0.409      5402
weighted avg      0.810     0.448     0.517      5402



In [204]:
#use a decision tree
from sklearn.tree import DecisionTreeClassifier

# Initialize the decision tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Train the model
dt.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_DT = dt.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_DT))

offByOne(y_test_standard, y_pred_DT)

              precision    recall  f1-score   support

           0       0.18      0.21      0.19       525
           1       0.19      0.50      0.28       199
           2       0.11      0.45      0.18       136
           3       0.06      0.34      0.11       134
           4       0.05      0.24      0.08       124
           5       0.10      0.24      0.14       275
           6       0.37      0.42      0.39       724
           7       0.73      0.21      0.32      3285

    accuracy                           0.26      5402
   macro avg       0.23      0.32      0.21      5402
weighted avg       0.53      0.26      0.29      5402

              precision    recall  f1-score   support

           0      0.623     0.410     0.494       525
           1      0.287     0.724     0.411       199
           2      0.164     0.640     0.260       136
           3      0.106     0.582     0.179       134
           4      0.116     0.540     0.191       124
           5      0.227 

In [205]:
# use a random forest classifier
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=1000, random_state=42)

# Train the model
rf.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_RF = rf.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_RF,digits=3))

print(offByOne(y_test_standard, y_pred_RF))

              precision    recall  f1-score   support

           0      0.209     0.354     0.263       525
           1      0.227     0.623     0.332       199
           2      0.114     0.618     0.192       136
           3      0.064     0.410     0.110       134
           4      0.102     0.387     0.161       124
           5      0.162     0.211     0.183       275
           6      0.378     0.576     0.457       724
           7      0.729     0.096     0.169      3285

    accuracy                          0.238      5402
   macro avg      0.248     0.409     0.233      5402
weighted avg      0.537     0.238     0.222      5402

              precision    recall  f1-score   support

           0      0.574     0.459     0.510       525
           1      0.296     0.814     0.434       199
           2      0.154     0.816     0.259       136
           3      0.109     0.709     0.189       134
           4      0.181     0.645     0.282       124
           5      0.355 

In [206]:
# use a bagging classifier
from sklearn.ensemble import BaggingClassifier
base_estimator = RandomForestClassifier(n_estimators=100,  random_state=42)

bagging = BaggingClassifier(base_estimator=base_estimator, n_estimators=10, random_state=42)

# Train the model
bagging.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_bagging = bagging.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_bagging,digits=3))

print(offByOne(y_test_standard, y_pred_bagging))

              precision    recall  f1-score   support

           0      0.207     0.358     0.262       525
           1      0.229     0.623     0.335       199
           2      0.110     0.610     0.186       136
           3      0.068     0.440     0.117       134
           4      0.110     0.411     0.173       124
           5      0.154     0.211     0.178       275
           6      0.375     0.565     0.451       724
           7      0.740     0.088     0.158      3285

    accuracy                          0.234      5402
   macro avg      0.249     0.413     0.233      5402
weighted avg      0.543     0.234     0.215      5402

              precision    recall  f1-score   support

           0      0.563     0.450     0.500       525
           1      0.306     0.834     0.448       199
           2      0.153     0.831     0.258       136
           3      0.112     0.746     0.195       134
           4      0.179     0.613     0.277       124
           5      0.346 

# Remove worst 5 features

In [207]:
def clipAndNormalize(features):
    #clip the features to the range of the training data
    features.drop(['key','time_signature','mode', 'tempo','liveness'], axis=1, inplace=True)
    #clip outliers to 1st and 99th percentile
    features['danceability'] = features['danceability'].clip(lower=features['danceability'].quantile(0.01), upper=features['danceability'].quantile(0.99))
    features['energy'] = features['energy'].clip(lower=features['energy'].quantile(0.01), upper=features['energy'].quantile(0.99))
    features['loudness'] = features['loudness'].clip(lower=features['loudness'].quantile(0.01), upper=features['loudness'].quantile(0.99))
    features['speechiness'] = features['speechiness'].clip(lower=features['speechiness'].quantile(0.01), upper=features['speechiness'].quantile(0.99))
    features['acousticness'] = features['acousticness'].clip(lower=features['acousticness'].quantile(0.01), upper=features['acousticness'].quantile(0.99))
    features['instrumentalness'] = features['instrumentalness'].clip(lower=features['instrumentalness'].quantile(0.01), upper=features['instrumentalness'].quantile(0.99))
    #features['liveness'] = features['liveness'].clip(lower=features['liveness'].quantile(0.01), upper=features['liveness'].quantile(0.99))
    features['valence'] = features['valence'].clip(lower=features['valence'].quantile(0.01), upper=features['valence'].quantile(0.99))
    #features['tempo'] = features['tempo'].clip(lower=features['tempo'].quantile(0.01), upper=features['tempo'].quantile(0.99))
    features['duration_ms'] = features['duration_ms'].clip(lower=features['duration_ms'].quantile(0.01), upper=features['duration_ms'].quantile(0.99))
    #features['time_signature'] = features['time_signature'].clip(lower=features['time_signature'].quantile(0.01), upper=features['time_signature'].quantile(0.99))

    columns_to_log=[ 'instrumentalness', 'acousticness', 'speechiness','loudness','energy']

    for i in columns_to_log:
        if i == 'loudness':
            features[i] = features[i] + 60
        features[i] = np.log(features[i]+1)

    
    #normalize the data
    scaler = StandardScaler()

    #if id is a column, drop it
    if 'id' in features.columns:
        #fit on all columns except the track id
        rawfeatures = features.drop(['id'], axis=1)
    else:
        rawfeatures = features
    preprocessedFeatures = scaler.fit_transform(rawfeatures)

    preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    #apply z-score normalization
    for i in columns_to_log:
        preprocessedFeaturesDF[i] = stats.zscore(preprocessedFeaturesDF[i])
        preprocessedFeaturesDF.clip(lower=-2.7, upper=2.7, inplace=True)



    #preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    '''#convert to dictionary, with track id as key
    preprocessedFeatures = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)
    preprocessedFeatures['id']= features['id']
    preprocessedFeatures = preprocessedFeatures.set_index('id').T.to_dict('list')'''
    return preprocessedFeaturesDF, preprocessedFeatures

In [208]:
def makeCategorical(df):
    mood_order=['sad','angry','energetic','excited','happy','content','calm','depressed']
    mood_codes, mood_categories = pd.factorize(mood_order)
    
    # Create a categorical object with the desired order
    cat = pd.Categorical(df['mood'], categories=mood_order, ordered=True)

    # Get the integer codes of the categories
    codes = cat.codes

    # Add the codes as a new column to the dataframe
    df['mood_code'] = codes
    return df



In [209]:
chdir('C:/Users/mlar5/OneDrive/Desktop/Code Folder/Python Projects/IRL projects/Aspire - Affective Computing Project/Playlists Data/Audio Features/emotion joint data')

In [210]:
emotionsDF = pd.read_csv('Merged Emotions Data5.csv')

In [211]:
emotionsDF = makeCategorical(emotionsDF)

In [212]:
emotionsDF['mood_code'].value_counts()

7    3779
6    1218
0    1020
5     770
1     694
2     630
3     628
4     618
Name: mood_code, dtype: int64

In [213]:
# create a new df with only up to 500 songs per mood_code
# this is to balance the data

balancedDF = pd.DataFrame(columns=emotionsDF.columns)

largest_balanced_sample = emotionsDF['mood_code'].value_counts().min()

for i in emotionsDF['mood_code'].unique():
    df = emotionsDF[emotionsDF['mood_code']==i]
    #if the value count of the mood_code is larger than 500, sample 500
    if df['mood_code'].value_counts()[i] >= largest_balanced_sample:
        df = df.sample(n=largest_balanced_sample, random_state=42)
    #if the value count of the mood_code is less than 500, sample the value count
    else:
        df = df.sample(n=df['mood_code'].value_counts()[i])
    balancedDF = pd.concat([balancedDF, df])

balancedDF['mood_code'].value_counts()

1    618
6    618
5    618
7    618
2    618
3    618
4    618
0    618
Name: mood_code, dtype: int64

In [214]:
balancedDF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song,mood,genre,mood_code
381,0.848,0.520,5,-10.663,0,0.5010,0.002250,0.000799,0.679,0.304,149.996,spotify:track:2XSrt1dcuOXPgl3B4bxmBz,203897,4,Carrollton,angry,rap,1
666,0.713,0.698,10,-7.435,0,0.1680,0.180000,0.000001,0.304,0.480,124.973,spotify:track:1SSv8SA2OHfOUwLgb8yOum,180062,4,Cheat Cxdes,angry,rap,1
257,0.757,0.423,1,-2.311,1,0.0527,0.000004,0.897000,0.118,0.125,130.058,spotify:track:0A8Mrg7ButLr17K3A0R61D,133308,4,TOTALITARIANISM,angry,EDM,1
338,0.516,0.515,1,-13.005,1,0.2790,0.033600,0.000002,0.119,0.396,95.971,spotify:track:583TaS41X2JJGKoGXnTY3l,107159,4,KILLTHEPHARAOH,angry,rap,1
319,0.618,0.836,6,-4.750,0,0.0813,0.002400,0.000000,0.363,0.397,175.060,spotify:track:7CMy59461Q3pgsPZ4Cj8CP,89143,4,EASE,angry,rap,1


In [215]:
rawfeatures = balancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)

In [216]:
rawfeaturesDF, rawfeatures = clipAndNormalize(rawfeatures)

In [217]:
rawfeaturesDF.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,valence,duration_ms
0,1.482419,-0.322342,-0.665392,2.700000,-0.958541,-0.489430,-0.552893,0.272427
1,0.545890,0.371919,0.064003,0.657598,-0.245604,-0.492971,0.192677,-0.199505
2,0.851129,-0.735754,1.136073,-0.530916,-0.967633,2.350764,-1.311172,-1.125231
3,-0.820748,-0.342998,-1.224490,1.695751,-0.824045,-0.492970,-0.163163,-1.642980
4,-0.113148,0.861786,0.637965,-0.224387,-0.957888,-0.492977,-0.158927,-1.999696


In [218]:
y = balancedDF['mood_code']

In [219]:
#set it to categorical
y = y.astype('category')

In [220]:
#X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#make a new dataframe of all the rows in emotionsDF that are not in balancedDF
unbalancedDF = emotionsDF[~emotionsDF.index.isin(balancedDF.index)]
extra_test_features = unbalancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)
extra_test_features_DF, extra_test_features = clipAndNormalize(extra_test_features)
extra_test_y = unbalancedDF['mood_code'].astype('category')
X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#combine the extra test features to the test set numpy arrays
X_test_standard = np.concatenate((X_test_standard, extra_test_features), axis=0)
y_test_standard = np.concatenate((y_test_standard, extra_test_y), axis=0)



In [221]:

# Initialize the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(256,128),random_state=42,early_stopping=True)

In [222]:
# Train the model on the resampled data
mlp.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred = mlp.predict(X_test_standard)

# Evaluate the model performance of micro-averaged F1 score

print(classification_report(y_test_standard, y_pred,digits=3))


              precision    recall  f1-score   support

           0      0.208     0.450     0.285       525
           1      0.237     0.523     0.326       199
           2      0.090     0.654     0.159       136
           3      0.074     0.313     0.119       134
           4      0.107     0.444     0.173       124
           5      0.162     0.233     0.191       275
           6      0.451     0.539     0.491       724
           7      0.796     0.122     0.212      3285

    accuracy                          0.256      5402
   macro avg      0.266     0.410     0.245      5402
weighted avg      0.588     0.256     0.255      5402



In [223]:
def offByOne(y_test_standard, y_pred,digits=3):
    #compare y_test_standard with y_pred_list. If y_pred_list is +-1 from y_test_standard, then it change it to be the same as y_test_standard
    y_test_standard_list=list(y_test_standard)
    y_pred_list = list(y_pred)
    for id in range(len(y_test_standard_list)):
        if y_test_standard_list[id] != 0 and y_test_standard_list[id] != 7:
            if y_pred_list[id] == y_test_standard_list[id] - 1 or y_pred_list[id] == y_test_standard_list[id] + 1:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 0:
            if y_pred_list[id] ==  1 or y_pred_list[id] == 7:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 7:
            if y_pred_list[id] ==  0 or y_pred_list[id] == 6:
                y_pred_list[id] = y_test_standard_list[id]
    print(classification_report(y_test_standard_list, y_pred_list,digits = digits))
    return

In [224]:
offByOne(y_test_standard, y_pred)

              precision    recall  f1-score   support

           0      0.530     0.510     0.520       525
           1      0.357     0.839     0.501       199
           2      0.116     0.794     0.202       136
           3      0.149     0.679     0.245       134
           4      0.146     0.532     0.229       124
           5      0.332     0.436     0.377       275
           6      0.960     0.727     0.827       724
           7      0.976     0.453     0.619      3285

    accuracy                          0.525      5402
   macro avg      0.446     0.621     0.440      5402
weighted avg      0.814     0.525     0.592      5402



In [225]:
#svm = SVC(kernel='linear', class_weight='balanced', random_state=42)
svm =SVC(kernel='poly', degree=3,class_weight='balanced', random_state=42)


In [226]:
# Train the model
svm.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_SVM = svm.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_SVM,digits=3))

              precision    recall  f1-score   support

           0      0.193     0.310     0.238       525
           1      0.304     0.487     0.375       199
           2      0.100     0.647     0.173       136
           3      0.083     0.403     0.137       134
           4      0.070     0.685     0.126       124
           5      0.099     0.175     0.126       275
           6      0.519     0.285     0.368       724
           7      0.782     0.143     0.242      3285

    accuracy                          0.224      5402
   macro avg      0.269     0.392     0.223      5402
weighted avg      0.587     0.224     0.251      5402



In [227]:
offByOne(y_test_standard, y_pred_SVM)

              precision    recall  f1-score   support

           0      0.496     0.350     0.411       525
           1      0.433     0.764     0.553       199
           2      0.124     0.765     0.214       136
           3      0.155     0.806     0.260       134
           4      0.082     0.726     0.148       124
           5      0.348     0.542     0.424       275
           6      0.956     0.626     0.756       724
           7      0.974     0.341     0.505      3285

    accuracy                          0.437      5402
   macro avg      0.446     0.615     0.409      5402
weighted avg      0.811     0.437     0.505      5402



In [228]:
#use a decision tree
from sklearn.tree import DecisionTreeClassifier

# Initialize the decision tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Train the model
dt.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_DT = dt.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_DT))

offByOne(y_test_standard, y_pred_DT)

              precision    recall  f1-score   support

           0       0.17      0.21      0.18       525
           1       0.21      0.50      0.30       199
           2       0.10      0.43      0.16       136
           3       0.06      0.33      0.10       134
           4       0.06      0.26      0.09       124
           5       0.12      0.24      0.16       275
           6       0.36      0.40      0.38       724
           7       0.72      0.22      0.34      3285

    accuracy                           0.26      5402
   macro avg       0.22      0.32      0.21      5402
weighted avg       0.52      0.26      0.30      5402

              precision    recall  f1-score   support

           0      0.599     0.410     0.486       525
           1      0.304     0.709     0.425       199
           2      0.150     0.640     0.243       136
           3      0.100     0.552     0.169       134
           4      0.125     0.556     0.204       124
           5      0.266 

In [229]:
# use a random forest classifier
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=1000, random_state=42)

# Train the model
rf.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_RF = rf.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_RF,digits=3))

print(offByOne(y_test_standard, y_pred_RF))

              precision    recall  f1-score   support

           0      0.209     0.339     0.258       525
           1      0.223     0.618     0.328       199
           2      0.113     0.618     0.192       136
           3      0.065     0.425     0.113       134
           4      0.104     0.395     0.164       124
           5      0.173     0.233     0.198       275
           6      0.380     0.576     0.458       724
           7      0.723     0.096     0.170      3285

    accuracy                          0.238      5402
   macro avg      0.249     0.413     0.235      5402
weighted avg      0.535     0.238     0.223      5402

              precision    recall  f1-score   support

           0      0.578     0.450     0.506       525
           1      0.298     0.829     0.438       199
           2      0.154     0.816     0.259       136
           3      0.108     0.724     0.188       134
           4      0.168     0.589     0.262       124
           5      0.357 

In [230]:
# use a bagging classifier
from sklearn.ensemble import BaggingClassifier
base_estimator = RandomForestClassifier(n_estimators=100,  random_state=42)

bagging = BaggingClassifier(base_estimator=base_estimator, n_estimators=10, random_state=42)

# Train the model
bagging.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_bagging = bagging.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_bagging,digits=3))

print(offByOne(y_test_standard, y_pred_bagging))

              precision    recall  f1-score   support

           0      0.213     0.352     0.266       525
           1      0.233     0.618     0.339       199
           2      0.109     0.618     0.185       136
           3      0.065     0.440     0.114       134
           4      0.104     0.395     0.164       124
           5      0.169     0.229     0.195       275
           6      0.381     0.577     0.459       724
           7      0.742     0.089     0.159      3285

    accuracy                          0.236      5402
   macro avg      0.252     0.415     0.235      5402
weighted avg      0.547     0.236     0.218      5402

              precision    recall  f1-score   support

           0      0.578     0.446     0.503       525
           1      0.310     0.824     0.451       199
           2      0.149     0.824     0.252       136
           3      0.110     0.746     0.191       134
           4      0.180     0.629     0.280       124
           5      0.365 

# Remove all but top 5 features

In [231]:
def clipAndNormalize(features):
    #clip the features to the range of the training data
    features.drop(['key', 'speechiness','danceability','duration_ms','liveness','tempo','mode','time_signature'], axis=1, inplace=True)
    #clip outliers to 1st and 99th percentile
    #features['danceability'] = features['danceability'].clip(lower=features['danceability'].quantile(0.01), upper=features['danceability'].quantile(0.99))
    features['energy'] = features['energy'].clip(lower=features['energy'].quantile(0.01), upper=features['energy'].quantile(0.99))
    features['loudness'] = features['loudness'].clip(lower=features['loudness'].quantile(0.01), upper=features['loudness'].quantile(0.99))
    #features['speechiness'] = features['speechiness'].clip(lower=features['speechiness'].quantile(0.01), upper=features['speechiness'].quantile(0.99))
    features['acousticness'] = features['acousticness'].clip(lower=features['acousticness'].quantile(0.01), upper=features['acousticness'].quantile(0.99))
    features['instrumentalness'] = features['instrumentalness'].clip(lower=features['instrumentalness'].quantile(0.01), upper=features['instrumentalness'].quantile(0.99))
    #features['liveness'] = features['liveness'].clip(lower=features['liveness'].quantile(0.01), upper=features['liveness'].quantile(0.99))
    features['valence'] = features['valence'].clip(lower=features['valence'].quantile(0.01), upper=features['valence'].quantile(0.99))
    #features['tempo'] = features['tempo'].clip(lower=features['tempo'].quantile(0.01), upper=features['tempo'].quantile(0.99))
    #features['duration_ms'] = features['duration_ms'].clip(lower=features['duration_ms'].quantile(0.01), upper=features['duration_ms'].quantile(0.99))
    #features['time_signature'] = features['time_signature'].clip(lower=features['time_signature'].quantile(0.01), upper=features['time_signature'].quantile(0.99))

    columns_to_log=[ 'instrumentalness', 'acousticness', 'loudness','energy']

    for i in columns_to_log:
        if i == 'loudness':
            features[i] = features[i] + 60
        features[i] = np.log(features[i]+1)

    
    #normalize the data
    scaler = StandardScaler()

    #if id is a column, drop it
    if 'id' in features.columns:
        #fit on all columns except the track id
        rawfeatures = features.drop(['id'], axis=1)
    else:
        rawfeatures = features
    preprocessedFeatures = scaler.fit_transform(rawfeatures)

    preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    #apply z-score normalization
    for i in columns_to_log:
        preprocessedFeaturesDF[i] = stats.zscore(preprocessedFeaturesDF[i])
        preprocessedFeaturesDF.clip(lower=-2.7, upper=2.7, inplace=True)



    #preprocessedFeaturesDF = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)

    '''#convert to dictionary, with track id as key
    preprocessedFeatures = pd.DataFrame(preprocessedFeatures, columns=rawfeatures.columns)
    preprocessedFeatures['id']= features['id']
    preprocessedFeatures = preprocessedFeatures.set_index('id').T.to_dict('list')'''
    return preprocessedFeaturesDF, preprocessedFeatures

In [232]:
def makeCategorical(df):
    mood_order=['sad','angry','energetic','excited','happy','content','calm','depressed']
    mood_codes, mood_categories = pd.factorize(mood_order)
    
    # Create a categorical object with the desired order
    cat = pd.Categorical(df['mood'], categories=mood_order, ordered=True)

    # Get the integer codes of the categories
    codes = cat.codes

    # Add the codes as a new column to the dataframe
    df['mood_code'] = codes
    return df



In [233]:
chdir('C:/Users/mlar5/OneDrive/Desktop/Code Folder/Python Projects/IRL projects/Aspire - Affective Computing Project/Playlists Data/Audio Features/emotion joint data')

In [234]:
emotionsDF = pd.read_csv('Merged Emotions Data5.csv')

In [235]:
emotionsDF = makeCategorical(emotionsDF)

In [236]:
emotionsDF['mood_code'].value_counts()

7    3779
6    1218
0    1020
5     770
1     694
2     630
3     628
4     618
Name: mood_code, dtype: int64

In [237]:
# create a new df with only up to 500 songs per mood_code
# this is to balance the data

balancedDF = pd.DataFrame(columns=emotionsDF.columns)

largest_balanced_sample = emotionsDF['mood_code'].value_counts().min()

for i in emotionsDF['mood_code'].unique():
    df = emotionsDF[emotionsDF['mood_code']==i]
    #if the value count of the mood_code is larger than 500, sample 500
    if df['mood_code'].value_counts()[i] >= largest_balanced_sample:
        df = df.sample(n=largest_balanced_sample, random_state=42)
    #if the value count of the mood_code is less than 500, sample the value count
    else:
        df = df.sample(n=df['mood_code'].value_counts()[i])
    balancedDF = pd.concat([balancedDF, df])

balancedDF['mood_code'].value_counts()

1    618
6    618
5    618
7    618
2    618
3    618
4    618
0    618
Name: mood_code, dtype: int64

In [238]:
balancedDF.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song,mood,genre,mood_code
381,0.848,0.520,5,-10.663,0,0.5010,0.002250,0.000799,0.679,0.304,149.996,spotify:track:2XSrt1dcuOXPgl3B4bxmBz,203897,4,Carrollton,angry,rap,1
666,0.713,0.698,10,-7.435,0,0.1680,0.180000,0.000001,0.304,0.480,124.973,spotify:track:1SSv8SA2OHfOUwLgb8yOum,180062,4,Cheat Cxdes,angry,rap,1
257,0.757,0.423,1,-2.311,1,0.0527,0.000004,0.897000,0.118,0.125,130.058,spotify:track:0A8Mrg7ButLr17K3A0R61D,133308,4,TOTALITARIANISM,angry,EDM,1
338,0.516,0.515,1,-13.005,1,0.2790,0.033600,0.000002,0.119,0.396,95.971,spotify:track:583TaS41X2JJGKoGXnTY3l,107159,4,KILLTHEPHARAOH,angry,rap,1
319,0.618,0.836,6,-4.750,0,0.0813,0.002400,0.000000,0.363,0.397,175.060,spotify:track:7CMy59461Q3pgsPZ4Cj8CP,89143,4,EASE,angry,rap,1


In [239]:
rawfeatures = balancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)

In [240]:
rawfeaturesDF, rawfeatures = clipAndNormalize(rawfeatures)

In [241]:
rawfeaturesDF.head()

,energy,loudness,acousticness,instrumentalness,valence
0,-0.322342,-0.665392,-0.958541,-0.489430,-0.552893
1,0.371919,0.064003,-0.245604,-0.492971,0.192677
2,-0.735754,1.136073,-0.967633,2.350764,-1.311172
3,-0.342998,-1.224490,-0.824045,-0.492970,-0.163163
4,0.861786,0.637965,-0.957888,-0.492977,-0.158927


In [242]:
y = balancedDF['mood_code']

In [243]:
#set it to categorical
y = y.astype('category')

In [244]:
#make a new dataframe of all the rows in emotionsDF that are not in balancedDF
unbalancedDF = emotionsDF[~emotionsDF.index.isin(balancedDF.index)]
extra_test_features = unbalancedDF.drop(['uri', 'song','mood','genre','mood_code'], axis=1)
extra_test_features_DF, extra_test_features = clipAndNormalize(extra_test_features)
extra_test_y = unbalancedDF['mood_code'].astype('category')
X_train_standard, X_test_standard, y_train_standard, y_test_standard = train_test_split(rawfeatures, y, test_size=0.2, random_state=42, stratify=y)
#combine the extra test features to the test set numpy arrays
X_test_standard = np.concatenate((X_test_standard, extra_test_features), axis=0)
y_test_standard = np.concatenate((y_test_standard, extra_test_y), axis=0)



In [245]:

# Initialize the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(256,128),random_state=42,early_stopping=True)

In [246]:
# Train the model on the resampled data
mlp.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred = mlp.predict(X_test_standard)

# Evaluate the model performance of micro-averaged F1 score

print(classification_report(y_test_standard, y_pred,digits=3))


              precision    recall  f1-score   support

           0      0.190     0.320     0.239       525
           1      0.082     0.166     0.110       199
           2      0.097     0.691     0.170       136
           3      0.052     0.493     0.095       134
           4      0.069     0.121     0.088       124
           5      0.166     0.193     0.178       275
           6      0.373     0.584     0.455       724
           7      0.725     0.048     0.090      3285

    accuracy                          0.187      5402
   macro avg      0.219     0.327     0.178      5402
weighted avg      0.526     0.187     0.161      5402



In [247]:
def offByOne(y_test_standard, y_pred,digits=3):
    #compare y_test_standard with y_pred_list. If y_pred_list is +-1 from y_test_standard, then it change it to be the same as y_test_standard
    y_test_standard_list=list(y_test_standard)
    y_pred_list = list(y_pred)
    for id in range(len(y_test_standard_list)):
        if y_test_standard_list[id] != 0 and y_test_standard_list[id] != 7:
            if y_pred_list[id] == y_test_standard_list[id] - 1 or y_pred_list[id] == y_test_standard_list[id] + 1:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 0:
            if y_pred_list[id] ==  1 or y_pred_list[id] == 7:
                y_pred_list[id] = y_test_standard_list[id]
        elif y_test_standard_list[id] == 7:
            if y_pred_list[id] ==  0 or y_pred_list[id] == 6:
                y_pred_list[id] = y_test_standard_list[id]
    print(classification_report(y_test_standard_list, y_pred_list,digits = digits))
    return

In [248]:
offByOne(y_test_standard, y_pred)

              precision    recall  f1-score   support

           0      0.518     0.390     0.445       525
           1      0.276     0.628     0.383       199
           2      0.134     0.882     0.233       136
           3      0.080     0.739     0.145       134
           4      0.285     0.524     0.369       124
           5      0.355     0.371     0.363       275
           6      0.940     0.735     0.825       724
           7      0.985     0.403     0.572      3285

    accuracy                          0.476      5402
   macro avg      0.447     0.584     0.417      5402
weighted avg      0.816     0.476     0.552      5402



In [249]:
#svm = SVC(kernel='linear', class_weight='balanced', random_state=42)
svm =SVC(kernel='poly', degree=3,class_weight='balanced', random_state=42)


In [250]:
# Train the model
svm.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_SVM = svm.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_SVM,digits=3))

              precision    recall  f1-score   support

           0      0.156     0.257     0.194       525
           1      0.079     0.121     0.096       199
           2      0.083     0.596     0.145       136
           3      0.046     0.493     0.084       134
           4      0.070     0.323     0.114       124
           5      0.099     0.127     0.112       275
           6      0.470     0.209     0.289       724
           7      0.711     0.121     0.206      3285

    accuracy                          0.172      5402
   macro avg      0.214     0.281     0.155      5402
weighted avg      0.523     0.172     0.200      5402



In [251]:
offByOne(y_test_standard, y_pred_SVM)

              precision    recall  f1-score   support

           0      0.400     0.305     0.346       525
           1      0.327     0.583     0.419       199
           2      0.131     0.890     0.228       136
           3      0.083     0.888     0.151       134
           4      0.153     0.677     0.250       124
           5      0.297     0.284     0.290       275
           6      0.962     0.591     0.732       724
           7      0.991     0.310     0.473      3285

    accuracy                          0.393      5402
   macro avg      0.418     0.566     0.361      5402
weighted avg      0.807     0.393     0.465      5402



In [252]:
#use a decision tree
from sklearn.tree import DecisionTreeClassifier

# Initialize the decision tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Train the model
dt.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_DT = dt.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_DT))

offByOne(y_test_standard, y_pred_DT)

              precision    recall  f1-score   support

           0       0.15      0.22      0.18       525
           1       0.08      0.24      0.12       199
           2       0.11      0.45      0.17       136
           3       0.05      0.23      0.08       134
           4       0.06      0.27      0.10       124
           5       0.07      0.19      0.11       275
           6       0.36      0.38      0.37       724
           7       0.66      0.17      0.27      3285

    accuracy                           0.22      5402
   macro avg       0.19      0.27      0.17      5402
weighted avg       0.47      0.22      0.25      5402

              precision    recall  f1-score   support

           0      0.617     0.461     0.528       525
           1      0.191     0.573     0.286       199
           2      0.176     0.721     0.282       136
           3      0.093     0.455     0.155       134
           4      0.135     0.556     0.218       124
           5      0.186 

In [253]:
# use a random forest classifier
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=1000, random_state=42)

# Train the model
rf.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_RF = rf.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_RF,digits=3))

print(offByOne(y_test_standard, y_pred_RF))

              precision    recall  f1-score   support

           0      0.201     0.310     0.244       525
           1      0.101     0.362     0.158       199
           2      0.135     0.632     0.222       136
           3      0.060     0.388     0.104       134
           4      0.094     0.298     0.143       124
           5      0.123     0.182     0.147       275
           6      0.373     0.529     0.437       724
           7      0.651     0.108     0.185      3285

    accuracy                          0.222      5402
   macro avg      0.217     0.351     0.205      5402
weighted avg      0.482     0.222     0.220      5402

              precision    recall  f1-score   support

           0      0.634     0.518     0.570       525
           1      0.184     0.638     0.286       199
           2      0.179     0.794     0.292       136
           3      0.104     0.687     0.181       134
           4      0.181     0.540     0.271       124
           5      0.296 

In [254]:
# use a bagging classifier
from sklearn.ensemble import BaggingClassifier
base_estimator = RandomForestClassifier(n_estimators=100,  random_state=42)

bagging = BaggingClassifier(base_estimator=base_estimator, n_estimators=10, random_state=42)

# Train the model
bagging.fit(X_train_standard, y_train_standard)

# Make predictions on the test set
y_pred_bagging = bagging.predict(X_test_standard)

# Evaluate the model performance
print(classification_report(y_test_standard, y_pred_bagging,digits=3))

print(offByOne(y_test_standard, y_pred_bagging))

              precision    recall  f1-score   support

           0      0.191     0.318     0.239       525
           1      0.106     0.372     0.164       199
           2      0.134     0.625     0.221       136
           3      0.059     0.396     0.103       134
           4      0.096     0.290     0.145       124
           5      0.148     0.185     0.165       275
           6      0.380     0.570     0.456       724
           7      0.687     0.102     0.178      3285

    accuracy                          0.225      5402
   macro avg      0.225     0.357     0.209      5402
weighted avg      0.506     0.225     0.218      5402

              precision    recall  f1-score   support

           0      0.612     0.493     0.546       525
           1      0.189     0.648     0.293       199
           2      0.183     0.816     0.298       136
           3      0.103     0.701     0.180       134
           4      0.184     0.516     0.271       124
           5      0.343 